In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shutil
import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint

In [75]:
train_datagen = ImageDataGenerator(rescale=1./255.)

valid_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_directory(directory=r'./Data/done_images/train',
                                                   target_size=(50, 50),
                                                   batch_size=32,
                                                   class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(directory=r'./Data/done_images/valid',
                                                   target_size=(50, 50),
                                                   batch_size=32,
                                                   class_mode='categorical')

Found 3259 images belonging to 3 classes.
Found 406 images belonging to 3 classes.


In [76]:
batch_size = 32
steps_per_epoch = 3259 // batch_size
validation_steps = 406 // batch_size

### Budowa modelu

In [77]:
model = Sequential()

model.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(50, 50, 3)))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(units=256, activation='relu'))
model.add(layers.Dense(units=64, activation='relu'))
model.add(layers.Dense(units=3, activation='softmax'))

In [78]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 48, 48, 16)        448       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 22, 22, 32)        4640      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 9, 9, 64)          18496     
_________________________________________________________________
flatten_5 (Flatten)          (None, 5184)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)              

In [79]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [80]:
filepath = 'best_model_weights.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

history = model.fit_generator(generator=train_generator,
                             steps_per_epoch=steps_per_epoch,
                             epochs=30,
                             validation_data=valid_generator,
                             validation_steps=validation_steps,
                             callbacks=[checkpoint])

Epoch 1/30
101/101 [==============================] - ETA: 0s - loss: 0.7606 - accuracy: 0.7657
Epoch 00001: val_accuracy improved from -inf to 0.78906, saving model to best_model_weights.hdf5
101/101 [==============================] - 5s 46ms/step - loss: 0.7606 - accuracy: 0.7657 - val_loss: 0.6099 - val_accuracy: 0.7891
Epoch 2/30
101/101 [==============================] - ETA: 0s - loss: 0.5664 - accuracy: 0.7936
Epoch 00002: val_accuracy improved from 0.78906 to 0.79167, saving model to best_model_weights.hdf5
101/101 [==============================] - 4s 39ms/step - loss: 0.5664 - accuracy: 0.7936 - val_loss: 0.5568 - val_accuracy: 0.7917
Epoch 3/30
101/101 [==============================] - ETA: 0s - loss: 0.5269 - accuracy: 0.7927
Epoch 00003: val_accuracy did not improve from 0.79167
101/101 [==============================] - 4s 39ms/step - loss: 0.5269 - accuracy: 0.7927 - val_loss: 0.5243 - val_accuracy: 0.7917
Epoch 4/30
101/101 [==============================] - ETA: 0s - 

Epoch 29/30
101/101 [==============================] - ETA: 0s - loss: 0.1972 - accuracy: 0.9386
Epoch 00029: val_accuracy did not improve from 0.94792
101/101 [==============================] - 4s 38ms/step - loss: 0.1972 - accuracy: 0.9386 - val_loss: 0.2057 - val_accuracy: 0.9427
Epoch 30/30
101/101 [==============================] - ETA: 0s - loss: 0.1928 - accuracy: 0.9411
Epoch 00030: val_accuracy did not improve from 0.94792
101/101 [==============================] - 4s 39ms/step - loss: 0.1928 - accuracy: 0.9411 - val_loss: 0.1958 - val_accuracy: 0.9479


### Wyświetlenie wykresów prezentujących zmiany accuracy oraz categorical_crossentropy (loss) dla zbioru treningowego i walidacyjnego w zależności od epoki

In [81]:
def plot_hist(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['accuracy'], name='accuracy', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_accuracy'], name='val_accuracy', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Accuracy vs. Val Accuracy', xaxis_title='Epoki', yaxis_title='Accuracy', yaxis_type='log')
    fig.show()

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['loss'], name='loss', mode='markers+lines'))
    fig.add_trace(go.Scatter(x=hist['epoch'], y=hist['val_loss'], name='val_loss', mode='markers+lines'))
    fig.update_layout(width=1000, height=500, title='Loss vs. Val Loss', xaxis_title='Epoki', yaxis_title='Loss', yaxis_type='log')
    fig.show()

plot_hist(history)

### Wczytanie najlepszych wag

In [82]:
model.load_weights('best_model_weights.hdf5')

In [83]:
test_datagen = ImageDataGenerator(rescale=1./255.)

test_generator = test_datagen.flow_from_directory(directory=r'./Data/done_images/test',
                                                   target_size=(50, 50),
                                                   batch_size=32,
                                                   class_mode='categorical',
                                                   shuffle=False)


y_prob = model.predict_generator(test_generator, test_generator.samples)
y_prob

Found 407 images belonging to 3 classes.


array([[0.17115737, 0.8256283 , 0.00321432],
       [0.05519306, 0.49332315, 0.45148376],
       [0.14394319, 0.2896056 , 0.56645125],
       ...,
       [0.02447893, 0.05348451, 0.9220366 ],
       [0.02455707, 0.06325298, 0.9121899 ],
       [0.04604281, 0.09320793, 0.8607493 ]], dtype=float32)

#### Wyświetlenie przewidzianych klas przez model dla zbioru testowego

In [84]:
y_pred = np.argmax(y_prob, axis=1)
y_pred

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,

In [85]:
predictions  = pd.DataFrame({'class': y_pred})
predictions

,class
0,1
1,1
2,2
3,1
4,1
...,...
402,2
403,2
404,2
405,2


#### Wyświetlenie prawdziwych klas dla zbioru testowego

In [86]:
y_true = test_generator.classes
y_true

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

#### Wyświetlenie przewidzianych klas przez model dla zbioru testowego

In [87]:
y_pred = predictions['class'].values
y_pred

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,

#### Wyświetlenie oznaczeń klas

In [88]:
test_generator.class_indices

{'mask_weared_incorrect': 0, 'with_mask': 1, 'without_mask': 2}

In [89]:
classes = list(test_generator.class_indices.keys())
classes

['mask_weared_incorrect', 'with_mask', 'without_mask']

#### Wyświetlenie macierzy konfuzji

In [90]:
cm = confusion_matrix(y_true, y_pred)
cm

array([[  0,  10,   2],
       [  0, 317,   6],
       [  0,   6,  66]], dtype=int64)

#### Wyświetlenie macierzy konfuzji w ładniejszej postaci

In [91]:
def plot_confusion_matrix(cm):
    # Mulitclass classification, 3 classes
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=classes, index=classes[::-1])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=800, height=800, title='Confusion Matrix', font_size=16)
    fig.show()

import plotly.figure_factory as ff
plot_confusion_matrix(cm)

#### Wyświetlenie raportu klasyfikacji

In [92]:
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

                       precision    recall  f1-score   support

mask_weared_incorrect       0.00      0.00      0.00        12
            with_mask       0.95      0.98      0.97       323
         without_mask       0.89      0.92      0.90        72

             accuracy                           0.94       407
            macro avg       0.61      0.63      0.62       407
         weighted avg       0.91      0.94      0.93       407



C:\Users\miasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



#### Sprawdzenie dokładności modelu

In [93]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true,y_pred)

0.941031941031941

# Budowa takiego samego modelu jak pierwszy ale tutaj na danych treningowych wykonana będzie augmentacja

In [94]:
train_datagen_with_augmentation = ImageDataGenerator(
    rotation_range=40,
    rescale=1./255.,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    fill_mode='nearest'
)


train_generator_with_augmentation = train_datagen.flow_from_directory(directory=r'./Data/done_images/train',
                                                   target_size=(50, 50),
                                                   batch_size=32,
                                                   class_mode='categorical')

Found 3259 images belonging to 3 classes.


In [95]:
model2 = Sequential()

model2.add(layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(50, 50, 3)))
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model2.add(layers.MaxPooling2D(pool_size=(2, 2)))
model2.add(layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model2.add(layers.Flatten())
model2.add(layers.Dense(units=256, activation='relu'))
model2.add(layers.Dense(units=64, activation='relu'))
model2.add(layers.Dense(units=3, activation='softmax'))

In [96]:
model2.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 48, 48, 16)        448       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 24, 24, 16)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 22, 22, 32)        4640      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 11, 11, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 9, 9, 64)          18496     
_________________________________________________________________
flatten_6 (Flatten)          (None, 5184)              0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)              

In [97]:
model2.compile(optimizer=optimizers.RMSprop(lr=1e-5),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

In [98]:
filepath2 = 'best_model2_weights.hdf5'
checkpoint2 = ModelCheckpoint(filepath=filepath2, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=True, 
                             mode='max')

history2 = model2.fit_generator(generator=train_generator_with_augmentation,
                             steps_per_epoch=steps_per_epoch,
                             epochs=30,
                             validation_data=valid_generator,
                             validation_steps=validation_steps,
                             callbacks=[checkpoint2])

Epoch 1/30
100/101 [============================>.] - ETA: 0s - loss: 0.6883 - accuracy: 0.7925
Epoch 00001: val_accuracy improved from -inf to 0.79948, saving model to best_model2_weights.hdf5
101/101 [==============================] - 5s 48ms/step - loss: 0.6864 - accuracy: 0.7933 - val_loss: 0.5632 - val_accuracy: 0.7995
Epoch 2/30
101/101 [==============================] - ETA: 0s - loss: 0.5438 - accuracy: 0.7933
Epoch 00002: val_accuracy did not improve from 0.79948
101/101 [==============================] - 4s 41ms/step - loss: 0.5438 - accuracy: 0.7933 - val_loss: 0.5393 - val_accuracy: 0.7969
Epoch 3/30
101/101 [==============================] - ETA: 0s - loss: 0.5093 - accuracy: 0.7933
Epoch 00003: val_accuracy did not improve from 0.79948
101/101 [==============================] - 4s 42ms/step - loss: 0.5093 - accuracy: 0.7933 - val_loss: 0.5037 - val_accuracy: 0.7917
Epoch 4/30
100/101 [============================>.] - ETA: 0s - loss: 0.4657 - accuracy: 0.7987
Epoch 00004:

Epoch 28/30
101/101 [==============================] - ETA: 0s - loss: 0.2071 - accuracy: 0.9365
Epoch 00028: val_accuracy did not improve from 0.94792
101/101 [==============================] - 4s 41ms/step - loss: 0.2071 - accuracy: 0.9365 - val_loss: 0.2108 - val_accuracy: 0.9375
Epoch 29/30
101/101 [==============================] - ETA: 0s - loss: 0.2035 - accuracy: 0.9355
Epoch 00029: val_accuracy did not improve from 0.94792
101/101 [==============================] - 4s 40ms/step - loss: 0.2035 - accuracy: 0.9355 - val_loss: 0.2002 - val_accuracy: 0.9401
Epoch 30/30
100/101 [============================>.] - ETA: 0s - loss: 0.2028 - accuracy: 0.9390
Epoch 00030: val_accuracy did not improve from 0.94792
101/101 [==============================] - 4s 40ms/step - loss: 0.2015 - accuracy: 0.9396 - val_loss: 0.1796 - val_accuracy: 0.9479


### Wyświetlenie wykresów prezentujących zmiany accuracy oraz categorical_crossentropy (loss) dla zbioru treningowego i walidacyjnego w zależności od epoki

In [99]:
plot_hist(history2)

### Wczytanie najlepszych wag

In [100]:
model2.load_weights('best_model2_weights.hdf5')

In [101]:
y_prob2 = model2.predict_generator(test_generator, test_generator.samples)
y_prob2

array([[0.23326403, 0.76149994, 0.00523598],
       [0.02471109, 0.542726  , 0.43256298],
       [0.18296343, 0.3296224 , 0.4874142 ],
       ...,
       [0.02031235, 0.09462642, 0.88506126],
       [0.02229127, 0.09822018, 0.8794886 ],
       [0.03494824, 0.10926966, 0.8557821 ]], dtype=float32)

#### Wyświetlenie przewidzianych klas przez model dla zbioru testowego

In [102]:
y_pred2 = np.argmax(y_prob2, axis=1)
y_pred2

array([1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1,

In [103]:
predictions2  = pd.DataFrame({'class': y_pred2})
predictions2

,class
0,1
1,1
2,2
3,1
4,1
...,...
402,2
403,2
404,2
405,2


#### Wyświetlenie prawdziwych klas dla zbioru testowego

In [104]:
y_true2 = test_generator.classes
y_true2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

#### Wyświetlenie oznaczeń klas

In [105]:
test_generator.class_indices

{'mask_weared_incorrect': 0, 'with_mask': 1, 'without_mask': 2}

#### Wyświetlenie macierzy konfuzji

In [106]:
cm2 = confusion_matrix(y_true2, y_pred2)
cm2

array([[  0,  10,   2],
       [  0, 318,   5],
       [  0,   7,  65]], dtype=int64)

#### Wyświetlenie macierzy konfuzji w ładniejszej postaci

In [107]:
plot_confusion_matrix(cm2)

#### Wyświetlenie raportu klasyfikacji

In [108]:
print(classification_report(y_true2, y_pred2, target_names=test_generator.class_indices.keys()))

                       precision    recall  f1-score   support

mask_weared_incorrect       0.00      0.00      0.00        12
            with_mask       0.95      0.98      0.97       323
         without_mask       0.90      0.90      0.90        72

             accuracy                           0.94       407
            macro avg       0.62      0.63      0.62       407
         weighted avg       0.91      0.94      0.93       407



C:\Users\miasa\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



#### Sprawdzenie dokładności modelu

In [109]:
accuracy_score(y_true2,y_pred2)

0.941031941031941